# Covid Sentiment Analysis
using Spacy and word2vec

In [ ]:
import numpy as np, pandas as pd, seaborn as sns, matplotlib.pyplot as plt, nltk

from sklearn.feature_extraction.text import ( CountVectorizer, TfidfVectorizer )
from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD

!pip install spacy
!python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_lg
!pip install gensim
!pip install vis
!pip install pyldavis

import spacy

import gensim
import gensim.corpora as corpora

import pyLDAvis
import pyLDAvis.gensim_models



from bs4 import BeautifulSoup
import re
from wordcloud import STOPWORDS


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2022-08-12 17:39:54.649312: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 7.3 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
2022-08-12 17:40:06.252741: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 587.7 MB 8.5 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
Looking in indexes: https://pypi.org/simple, https://us-pyth

In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/hsuecu/data_set/main/Corona_NLP_train.csv", encoding='latin')
#data = pd.read_csv('Corona_NLP_train.csv', encoding='latin')
"""
this encoding resolved issue with utf-8 encoder.
"""
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41157 entries, 0 to 41156
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   UserName       41157 non-null  int64 
 1   ScreenName     41157 non-null  int64 
 2   Location       32567 non-null  object
 3   TweetAt        41157 non-null  object
 4   OriginalTweet  41157 non-null  object
 5   Sentiment      41157 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.9+ MB


In [ ]:
data.head(7)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative
5,3804,48756,"ÃT: 36.319708,-82.363649",16-03-2020,As news of the regionÂs first confirmed COVID...,Positive
6,3805,48757,"35.926541,-78.753267",16-03-2020,Cashier at grocery store was sharing his insig...,Positive


**Discriptions and general perception**

- UserName
- ScreenName
> Both are numbers, hence useless
- Location
> This can be usefull but need formating
    - some use character of other language, they need to be dropped
    - some use GPS location, but resolving it to location name can be tedious
    - some use abbriviation like UK for united kingdom
- TweetAt
> Date data, hence no use
- OriginalTweet
> This may contain hashtags, URLs, Character from other laguages, numbers. URLs get removed, Hashtag text is scrapped, tweet in other language be removeda and numbers be removed including currency or percentages.
- Sentiment
> Sentiment of the tweet
    1. Extremely Positive->1
    1. Positive->1
    1. Neural->1
    1. Negative->0
    1. Extremely Negative->0

In [ ]:
data['Sentiment'].unique()

array(['Neutral', 'Positive', 'Extremely Negative', 'Negative',
       'Extremely Positive'], dtype=object)

## Tweet Text Filtering

In [ ]:
def tweetScrapper(s):
    s = re.sub(r"http[s]://[a-zA-z0-9.$-_.+!*'(),]*", '', s)
    s = re.sub(r"[^a-zA-Z @]", '', s)
    s = re.sub(r"@[a-zA-Z0-9]*", '', s)
    s = str.lower(s)
    s = str.strip(s)
    return s

def locationScrapper(s):
    s = str(s)
    s = re.sub(r"[^a-zA-Z ]",'', s)
    s = str.lower(s)
    s = str.strip(s)
    return s

def sentimentMapper(s):
    hmap={
        'Extremely Positive': 0,
        'Positive': 0,
        'Neutral': 0,
        'Negative': 1,
        'Extremely Negative': 1
    }
    if s in hmap:
        return hmap[s]
    else:
        raise Exception(f"sentiment string '{s}' not known to mapper")

In [ ]:
print(data['OriginalTweet'][0])

@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8


In [ ]:
try:
    data['Location'].fillna('*', inplace=True)
    data['Location'] = data['Location'].apply(locationScrapper)
    data['OriginalTweet'] = data['OriginalTweet'] + ' ' + data['Location']
except:
    pass

for i in ['UserName', 'ScreenName', 'Location', 'TweetAt']:
    try:
        del data[i]
    except:
        pass

data['OriginalTweet'] = data['OriginalTweet'].apply(tweetScrapper)
data['Sentiment'] = data['Sentiment'].apply(sentimentMapper)

In [ ]:
data.head()

,OriginalTweet,Sentiment
0,and and london,0
1,advice talk to your neighbours family to excha...,0
2,coronavirus australia woolworths to give elder...,0
3,my food stock is not the only one which is emp...,0
4,me ready to go at supermarket during the covid...,1


In [ ]:
sp = spacy.load('en_core_web_lg')

In [ ]:
data['OriginalTweet'] = data['OriginalTweet'].apply(lambda s: ''.join([i.lemma_+' ' for i in sp(s) if i.text not in STOPWORDS]))
data.dropna(inplace=True)

In [ ]:
data.head()

,OriginalTweet,Sentiment
0,london,0
1,advice talk neighbour family exchange phone nu...,0
2,coronavirus australia woolworth give elderly d...,0
3,food stock one emptyplease not panic will enou...,0
4,ready go supermarket covid outbreaknot m paran...,1


In [ ]:
doc_train = [sp(i).vector for i in data['OriginalTweet']]

In [ ]:
data_text = np.vstack(doc_train)
data_score = data['Sentiment']

## Model Training

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_text, data_score, random_state=42)

### Logistics Regression

In [ ]:
lr = LogisticRegression(penalty='l2', max_iter=1000, C=1, random_state=42)
lr.fit(X_train, y_train)

LogisticRegression(C=1, max_iter=1000, random_state=42)

In [ ]:
y_pred = lr.predict(X_train)
print(f'Train Score:{accuracy_score(y_train, y_pred)}')

Train Score:0.782939709074416


In [ ]:
print(classification_report(y_train, y_pred, target_names=['0' , '1']))

              precision    recall  f1-score   support

           0       0.80      0.87      0.83     19299
           1       0.75      0.63      0.69     11568

    accuracy                           0.78     30867
   macro avg       0.77      0.75      0.76     30867
weighted avg       0.78      0.78      0.78     30867



In [ ]:
y_pred = lr.predict(X_test)
print(f'Test Score:{accuracy_score(y_test, y_pred)}')

Test Score:0.7847424684159378


In [ ]:
print(classification_report(y_test, y_pred, target_names=['0', '1']))

              precision    recall  f1-score   support

           0       0.80      0.88      0.84      6460
           1       0.75      0.63      0.68      3830

    accuracy                           0.78     10290
   macro avg       0.78      0.75      0.76     10290
weighted avg       0.78      0.78      0.78     10290



### Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

RandomForestClassifier()

In [ ]:
y_pred = rf.predict(X_train)
print(f'Train Score:{accuracy_score(y_train, y_pred)}')

Train Score:0.9999352058833058


In [ ]:
print(classification_report(y_train, y_pred, target_names=['0', '1']))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     19299
           1       1.00      1.00      1.00     11568

    accuracy                           1.00     30867
   macro avg       1.00      1.00      1.00     30867
weighted avg       1.00      1.00      1.00     30867



In [ ]:
y_pred = rf.predict(X_test)
print(f'Test Score:{accuracy_score(y_test, y_pred)}')

Test Score:0.726044703595724


In [ ]:
print(classification_report(y_test, y_pred, target_names=['0', '1']))

              precision    recall  f1-score   support

           0       0.72      0.91      0.81      6460
           1       0.74      0.41      0.53      3830

    accuracy                           0.73     10290
   macro avg       0.73      0.66      0.67     10290
weighted avg       0.73      0.73      0.70     10290



### XGBoost Classifier

In [ ]:
from xgboost import XGBClassifier

/home/analogus/.local/lib/python3.8/site-packages/pkg_resources/__init__.py:122: PkgResourcesDeprecationWarning: 0.1.36ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/home/analogus/.local/lib/python3.8/site-packages/pkg_resources/__init__.py:122: PkgResourcesDeprecationWarning: 0.23ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(


In [ ]:
xgb = XGBClassifier()
xgb.fit(X_train, y_train)

/home/analogus/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:31:50] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=6,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [ ]:
y_pred = xgb.predict(X_train)
print(f'Train Score:{accuracy_score(y_train, y_pred)}')

Train Score:0.9763177503482684


In [ ]:
print(classification_report(y_train, y_pred, target_names=['0' ,'1']))

              precision    recall  f1-score   support

           1       0.99      0.98      0.98      4981
           2       0.96      0.98      0.97      8524
           3       0.99      0.96      0.97      5794
           4       0.97      0.97      0.97      7404
           5       0.99      0.99      0.99      4164

    accuracy                           0.98     30867
   macro avg       0.98      0.98      0.98     30867
weighted avg       0.98      0.98      0.98     30867



In [ ]:
y_pred = xgb.predict(X_test)
print(f'Test Score:{accuracy_score(y_test, y_pred)}')

Test Score:0.44538386783284745


In [ ]:
print(classification_report(y_test, y_pred, target_names=['0', '1']))

              precision    recall  f1-score   support

           1       0.54      0.43      0.48      1643
           2       0.40      0.51      0.45      2898
           3       0.52      0.46      0.49      1919
           4       0.38      0.41      0.40      2513
           5       0.51      0.36      0.42      1317

    accuracy                           0.45     10290
   macro avg       0.47      0.43      0.45     10290
weighted avg       0.46      0.45      0.45     10290



## Testing with TF_IDF and LSA sentiment extraction

*Reference* 
- *https://towardsdatascience.com/lovecraft-with-natural-language-processing-part-3-tf-idf-vectors-8c2d4df98621*
- *https://www.kaggle.com/code/dex314/tfidf-truncatedsvd-and-light-gbm/notebook*

In [ ]:
def foo(x):
    return[i for i in x.split(' ') if i != '']

In [ ]:
tfidf_vectorizer = \
TfidfVectorizer(input = 'content', tokenizer = (foo))

In [ ]:
svdT = TruncatedSVD(n_components=20)
svdTfit = svdT.fit_transform(tfidf_vectorizer.fit_transform(data['OriginalTweet']))

In [ ]:
data_train = pd.DataFrame(svdTfit, columns=[str(i+1) for i in range(len(svdTfit[0]))])

In [ ]:
data_score = data['Sentiment']
X_train, X_test, y_train, y_test = train_test_split(data_train, data_score, random_state=42)

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

RandomForestClassifier()

In [ ]:
y_pred = rf.predict(X_train)
print(f'Train Score:{accuracy_score(y_train, y_pred)}')

Train Score:0.9999352058833058


In [ ]:
print(classification_report(y_train, y_pred, target_names=['0', '1']))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     19299
           1       1.00      1.00      1.00     11568

    accuracy                           1.00     30867
   macro avg       1.00      1.00      1.00     30867
weighted avg       1.00      1.00      1.00     30867



In [ ]:
y_pred = rf.predict(X_test)
print(f'Test Score:{accuracy_score(y_test, y_pred)}')

Test Score:0.686977648202138


In [ ]:
print(classification_report(y_test, y_pred, target_names=['0', '1']))

              precision    recall  f1-score   support

           0       0.70      0.89      0.78      6460
           1       0.65      0.34      0.45      3830

    accuracy                           0.69     10290
   macro avg       0.67      0.62      0.61     10290
weighted avg       0.68      0.69      0.66     10290



## Testing with LDA for Topic Extraction

*Reference*

- *https://www.youtube.com/watch?v=T05t-SqKArY*
- *https://www.youtube.com/watch?v=BaM1uiCpj_E&t=2s*
- *https://www.youtube.com/watch?v=TKjjlp5_r7o*


In [ ]:
text_list = data['OriginalTweet']
text_list.apply(lambda x: [i for i in x.split(' ') if i != ''])

0                                                 [london]
1        [advice, talk, neighbour, family, exchange, ph...
2        [coronavirus, australia, woolworth, give, elde...
3        [food, stock, one, emptyplease, not, panic, wi...
4        [ready, go, supermarket, covid, outbreaknot, m...
                               ...                        
41152    [airline, pilot, offer, stock, supermarket, sh...
41153    [response, complaint, provide, cite, covid, re...
41154    [know, get, tough, ration, toilet, paper, coro...
41155    [wrong, smell, hand, sanitizer, start, turn, o...
41156    [well, newuse, rift, s, go, amazon, rn, althou...
Name: OriginalTweet, Length: 41157, dtype: object

In [ ]:
dictionary = corpora.Dictionary(text_list.apply(lambda x: [i for i in x.split(' ') if i != '']))
doc_term_matrix = [dictionary.doc2bow(rev) for rev in text_list.apply(lambda x: [i for i in x.split(' ') if i != ''])]

In [ ]:
LDA = gensim.models.ldamodel.LdaModel

lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=5, random_state=42,
                chunksize=100, passes=25, iterations=50)

In [ ]:
lda_model.print_topics()

[(0,
  '0.028*"store" + 0.026*"grocery" + 0.024*"covid" + 0.012*"worker" + 0.011*"work" + 0.011*"go" + 0.010*"make" + 0.010*"mask" + 0.010*"amp" + 0.010*"people"'),
 (1,
  '0.050*"price" + 0.044*"covid" + 0.029*"consumer" + 0.015*"oil" + 0.015*"will" + 0.014*"pandemic" + 0.011*"coronavirus" + 0.009*"impact" + 0.009*"market" + 0.008*"due"'),
 (2,
  '0.030*"food" + 0.024*"covid" + 0.016*"consumer" + 0.016*"we" + 0.012*"amp" + 0.011*"demand" + 0.011*"toiletpaper" + 0.010*"help" + 0.009*"coronavirus" + 0.009*"crisis"'),
 (3,
  '0.032*"new" + 0.028*"covid" + 0.026*"online" + 0.019*"shopping" + 0.014*"india" + 0.010*"united" + 0.010*"york" + 0.009*"lockdown" + 0.009*"coronavirus" + 0.009*"delivery"'),
 (4,
  '0.025*"sanitizer" + 0.024*"coronavirus" + 0.024*"supermarket" + 0.021*"hand" + 0.021*"covid" + 0.013*"food" + 0.013*"go" + 0.012*"people" + 0.012*"not" + 0.011*"coronaviru"')]